In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_conv_1 = tripple_conv(1, 48)
        self.down_conv_2 = tripple_conv(48, 96)
        self.down_conv_3 = tripple_conv(96, 192)

        self.up_trans_01 = nn.ConvTranspose2d(
            in_channels=192,
            out_channels=96,
            kernel_size=2,
            stride=2)
        
        self.up_conv_01 = tripple_conv(192, 96)
        
        self.up_trans_02= nn.ConvTranspose2d(
            in_channels=96,
            out_channels=48,
            kernel_size=2,
            stride=2)        
        
        self.up_conv_02 = tripple_conv(96, 48)
        
        #2nd temporal context module
    
        self.down_conv_11 = tripple_conv(144, 48)
        self.down_conv_12 = tripple_conv(48, 96)
        self.down_conv_13 = tripple_conv(96, 192)
        
        self.up_trans_11 = nn.ConvTranspose2d(
            in_channels=192,
            out_channels=96,
            kernel_size=2,
            stride=2)
        self.up_conv_11 = tripple_conv(192, 96)
        
        self.up_trans_12= nn.ConvTranspose2d(
            in_channels=96,
            out_channels=48,
            kernel_size=2,
            stride=2)         
        
        self.up_conv_12 = tripple_conv(96, 48)   
        
        #tripple head output 

        self.out_prob =  double_conv_sigmoid(48, 1)
        self.out_coor =  double_conv_tanh(48, 2)    
        self.out_uncer =  double_conv_sigmoid(48, 4)       

    def forward(self, image):
        x01 = self.down_conv_1(image)
        x02 =  self.max_pool_2x2(x01)
        print('1st UNet down sampling stage 1 dimension', x02.size())
        x03 = self.down_conv_2(x02)
        x04 =  self.max_pool_2x2(x03) 
        print('1st UNet down sampling stage 2 dimension', x04.size())
        x05 = self.down_conv_3(x04)
        print('1st UNet transfer dimension', x05.size())

        #decoder
        x11 = self.up_trans_01(x05)
        x12= self.up_conv_01(torch.cat([x11, x03], 1))
        print('1st UNet up sampling stage 1 dimension', x12.size())
        x13 = self.up_trans_02(x12)
        x14 = self.up_conv_02(torch.cat([x13, x01], 1))
        print('1st UNet up sampling stage 2 dimension', x14.size())
        print('x14', x14.size())

        return x14

    def temporal_context(self, image_t_min_1, image_t, image_t_plus_1):
        t_min_1 = self.forward(image_t_min_1)
        t = self.forward(image_t)
        t_plus_1 = self.forward(image_t_plus_1)
        output = torch.cat((t_min_1, t, t_plus_1), 1)
        return output

    def forward2(self, image_t_min_1, image_t, image_t_plus_1):
        image1 = self.temporal_context(image_t_min_1, image_t, image_t_plus_1)
        x21 = self.down_conv_11(image1) 
        x22 =  self.max_pool_2x2(x21) 
        print('2nd UNet down sampling stage 1 dimension', x22.size())
        x23 = self.down_conv_12(x22)    
        x24 =  self.max_pool_2x2(x23) 
        print('2nd UNet down sampling stage 2 dimension', x24.size())
        x25 = self.down_conv_13(x24)
        print('2nd UNet transfer dimension', x25.size())
        
        #decoder
        x31 = self.up_trans_11(x25)
        x32= self.up_conv_11(torch.cat([x31, x23], 1))
        print('2nd UNet up sampling stage 1 dimension', x32.size())
        x33 = self.up_trans_12(x32)
        x34 = self.up_conv_12(torch.cat([x33, x21], 1))                
        print('2nd UNet up sampling stage 2 dimension', x34.size())

        return x34   

    def tripple_head(self, image_t_min_1, image_t, image_t_plus_1):
        head_split= self.forward2(image_t_min_1, image_t, image_t_plus_1)
        print(head_split.size())
        out_probability = self.out_prob(head_split)
        out_coordinates = self.out_coor(head_split)
        out_uncertainty = self.out_uncer(head_split)
        
        print('size of out_probability = ', out_probability.size())
        print('size of out_coordinates = ', out_coordinates.size())
        print('size of out_uncertainty = ', out_uncertainty.size())
        
        return out_probability, out_coordinates, out_uncertainty
    